In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/wmt-test-data-2025/indicMT_testset2025_release/Category-1/Mizo/en-lus.txt
/kaggle/input/wmt-test-data-2025/indicMT_testset2025_release/Category-1/Mizo/lus-en.txt
/kaggle/input/wmt-test-data-2025/indicMT_testset2025_release/Category-1/Assamese/as-en.txt
/kaggle/input/wmt-test-data-2025/indicMT_testset2025_release/Category-1/Assamese/en-as.txt
/kaggle/input/wmt-test-data-2025/indicMT_testset2025_release/Category-1/Manipuri/en-mni.txt
/kaggle/input/wmt-test-data-2025/indicMT_testset2025_release/Category-1/Manipuri/mni-en.txt
/kaggle/input/wmt-test-data-2025/indicMT_testset2025_release/Category-1/Khasi/kha-en.txt
/kaggle/input/wmt-test-data-2025/indicMT_testset2025_release/Category-1/Khasi/en-kha.txt
/kaggle/input/wmt-test-data-2025/indicMT_testset2025_release/Category-1/Nyishi/njz-en.txt
/kaggle/input/wmt-test-data-2025/indicMT_testset2025_release/Category-1/Nyishi/en-njz.txt
/kaggle/input/wmt-test-data-2025/indicMT_testset2025_release/Category-2/Bodo/bodo-en.txt
/kaggle/inp

In [2]:
!pip install -q transformers datasets accelerate bitsandbytes peft evaluate sacrebleu
!pip install -U bitsandbytes
!pip install datasets pandas sacremoses fasttext sentence-transformers sentencepiece scikit-learn
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
!pip install spacy indic-transliteration
!python -m spacy download en_core_web_sm 
!pip install indic-nlp-library 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 27.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install evaluate

In [4]:
import torch
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM, 
                          Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq)
from transformers import BitsAndBytesConfig
from peft import LoraConfig, TaskType, get_peft_model
import evaluate 

2025-06-12 06:25:39.567073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749709539.736368      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749709539.790782      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
import pandas as pd

file_path = "/kaggle/input/eng-to-mizo/English-Mizo Traning Data 2025.xlsx"
df = pd.read_excel(file_path)

csv_path = "/kaggle/working/english_manipuri_training_data_2025.csv"
df.to_csv(csv_path, index=False)

print(f"CSV file saved to: {csv_path}") 

CSV file saved to: /kaggle/working/english_manipuri_training_data_2025.csv


In [17]:
import pandas as pd

# Load the existing file (replace 'your_file.csv' with your actual filename)
df = pd.read_csv("/kaggle/working/english_manipuri_training_data_2025.csv", header=None)

# Rename the columns
df.columns = ['English', 'Mizo']

# Save it back to a new CSV file (or overwrite the same file)
df.to_csv("english_mizo.csv", index=False)
df

,English,Mizo
0,"Now therefore , my son , obey my voice . Ar...","Chuvângin , ka thu hi ngaithla teh . Haran..."
1,He is the Lord our God ; his judgments are in ...,Ani chu Lalpa kan Pathian chu a ni a ; A thupê...
2,Come back soon,Lo kir thuai ang che
3,"Then they come back by boat to Caesareʹa, and ...","Tin lawngin Kaisaria-ah an kir a , Antiokei-a..."
4,"Of the tribe of Benjamin , Elidad the son of ...",Benjamina hnam aṭangin Kislona fapa Elidada ;
...,...,...
49995,"He said to them , "" What have I now done in ...","Ani chuan , “Nangnin in tih ṭhin ai chuan k..."
49996,"20 According to Acts 13 : 38 - 41, the apostl...","20 Tirhkohte 13:38-41-a mi angin , tirhkoh P..."
49997,"For example, rather than saying essentially t...","Entîr nân , in tina thu inang sawi ai chuan ..."
49998,Therefore her young men shall fall in her stre...,Chuta tlangvâlte chu kawtthlêrah an tlu ang a ...


In [18]:
df

,English,Mizo
0,"Now therefore , my son , obey my voice . Ar...","Chuvângin , ka thu hi ngaithla teh . Haran..."
1,He is the Lord our God ; his judgments are in ...,Ani chu Lalpa kan Pathian chu a ni a ; A thupê...
2,Come back soon,Lo kir thuai ang che
3,"Then they come back by boat to Caesareʹa, and ...","Tin lawngin Kaisaria-ah an kir a , Antiokei-a..."
4,"Of the tribe of Benjamin , Elidad the son of ...",Benjamina hnam aṭangin Kislona fapa Elidada ;
...,...,...
49995,"He said to them , "" What have I now done in ...","Ani chuan , “Nangnin in tih ṭhin ai chuan k..."
49996,"20 According to Acts 13 : 38 - 41, the apostl...","20 Tirhkohte 13:38-41-a mi angin , tirhkoh P..."
49997,"For example, rather than saying essentially t...","Entîr nân , in tina thu inang sawi ai chuan ..."
49998,Therefore her young men shall fall in her stre...,Chuta tlangvâlte chu kawtthlêrah an tlu ang a ...


In [19]:
from datasets import load_dataset, Dataset

# Load original CSV
dataset = load_dataset(
    'csv',
    data_files="english_mizo.csv",
    split="train"
)

# Reverse columns: Assamese ← English, English ← Assamese
#reversed_examples = [{"Manipuri": example["Manipuri"], "English": example["English"]} for example in dataset]

# Rebuild as a new Dataset with exactly ordered columns
#reversed_dataset = Dataset.from_list(reversed_examples)

subset = dataset.shuffle(seed=42).select(range(30000))

dataset_split = subset.train_test_split(test_size=0.03)
train_dataset = dataset_split["train"]
test_dataset = dataset_split["test"]

print(train_dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['English', 'Mizo'],
    num_rows: 990
})


In [33]:
test_dataset[0]

{'input_ids': [256047,
  540,
  202,
  7038,
  120221,
  8362,
  53513,
  146,
  202,
  10527,
  108,
  81602,
  146,
  108,
  216503,
  146,
  108,
  1212,
  4518,
  146,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'labels': [256047,
  250,
  9265,
  41276,
  172,
  146,
  6241,
  2655,
  253,
  172,
  146,
  13,
  17135,
  22005,
  17716,
  83444,
  2237,
  248067,
  26454,
  133909,
  8249,
  22048,
  47272,
  169,
  172,
  146,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

model_name = "facebook/nllb-200-3.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Set target language code (must match NLLB language codes)
tgt_lang =  "lus_Latn" 
forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)

# Configure 8-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True   
)

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    forced_bos_token_id=forced_bos_token_id,
    device_map="auto"  
) 

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

...

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/6.93G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/94.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [31]:
def preprocess_examples(examples):
    tokenizer.src_lang = "eng_Latn" 
    tokenizer.tgt_lang = "lus_Latn"

    # Remove examples where English or Mizo is not a string
    valid_pairs = [
        (eng, miz) for eng, miz in zip(examples["English"], examples["Mizo"])
        if isinstance(eng, str) and isinstance(miz, str)
    ]
    
    if not valid_pairs:
        return {}

    eng_texts, miz_texts = zip(*valid_pairs)

    model_inputs = tokenizer(
        list(eng_texts),
        max_length=256,
        padding="longest",
        truncation=True
    )

    labels = tokenizer(
        list(miz_texts),
        max_length=256,
        padding="longest",
        truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
  
train_dataset = train_dataset.map(preprocess_examples, batched=True, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(preprocess_examples, batched=True, remove_columns=test_dataset.column_names) 


Map:   0%|          | 0/990 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [32]:
len(train_dataset)

989

In [34]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=64,
    lora_alpha=128,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  
    #use_dora=True
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
# device = torch.device("cuda")

# model=model.to(device)


trainable params: 56,623,104 || all params: 3,401,486,336 || trainable%: 1.6647


In [36]:
import torch

assamese_text="i love my beautiful country where i belong to ."

tokenizer.src_lang = "eng_Latn" 
tokenizer.tgt_lang = "lus_Latn"

# Tokenize the input
inputs = tokenizer(
    assamese_text,
    return_tensors="pt",
    padding=True,
    truncation=True
).to(model.device)

# Generate output
translated_tokens = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    max_length=256
)

# Decode the output
translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

print("Translated Text:", translation)


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1667: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Translated Text: Ka awmna ram mawi tak hi ka hmangaih a ni .


In [37]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="longest",
    pad_to_multiple_of=8,
    label_pad_token_id=-100
)

In [38]:
bleu_metric = evaluate.load("sacrebleu")


In [39]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds.predictions, eval_preds.label_ids
    if isinstance(preds, tuple):
        preds = preds[0]
    if preds.ndim == 3:
        preds = np.argmax(preds, axis=-1)

    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
   
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}


In [40]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    learning_rate=3e-5, 
    warmup_steps=1000,
    lr_scheduler_type="cosine", 
    logging_steps=500,
    eval_strategy="epoch",
    save_strategy="epoch",
   # save_steps=1000,
    save_total_limit=2,
    fp16=True,
    predict_with_generate=True,
    report_to="none",
    optim="adamw_torch_fused",
    adam_beta1=0.9,
    adam_beta2=0.98,
    ddp_find_unused_parameters=False,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

model.config.use_cache = False


/tmp/ipykernel_35/4087804095.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [41]:
trainer.train() 

/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss,Bleu
1,No log,9.577372,24.437193


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1667: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


TrainOutput(global_step=124, training_loss=12.804874543220766, metrics={'train_runtime': 318.0643, 'train_samples_per_second': 3.109, 'train_steps_per_second': 0.39, 'total_flos': 1365663315394560.0, 'train_loss': 12.804874543220766, 'epoch': 1.0})

In [42]:
predictions = trainer.predict(test_dataset) 

import numpy as np

if isinstance(predictions.predictions, tuple):
    preds = predictions.predictions[0]
else:
    preds = predictions.predictions

if preds.ndim == 3:
    preds = np.argmax(preds, axis=-1)

labels = np.where(predictions.label_ids == -100, tokenizer.pad_token_id, predictions.label_ids)

decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

for ref, pred in zip(decoded_labels[:10], decoded_preds[:10]):
    print(f"Actual   : {ref}")
    print(f"Predicted: {pred}")
    print("-" * 100)


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Actual   : Rangkachak te , tangkarua te , dâr tea thil mawi tinrêng ruahmana siam thiamna te ,
Predicted: leh thil thiamna te , rangkachak te , tangkarua te , bronze te ,
----------------------------------------------------------------------------------------------------
Actual   : (Luka 13:24) Mahse , Bible chuan Sâm 126: 5-ah he thutiam min pe: Mittui tla chunga tuhte chuan hlim takin an ât ang ,  tiin .
Predicted: (Luka 13: 24) Mahse , Bible chuan Sâm 126: 5 - ah chuan heti hian a tiam a ni:  Mittuiin chi thehtute chuan hlim takin an seng ang .
----------------------------------------------------------------------------------------------------
Actual   : Chu kan unau chu kan zavai atâna hmêlma Setana a donaah kan ṭanpui a .
Predicted: Kan hmêlma Satan do tûrin kan unaupa chu kan ṭanpui a ni.
----------------------------------------------------------------------------------------------------
Actual   : Chu bâkah , hêng Juda upaten he Pathian in an sakna tûrah hian ṭanpui tûrin thu ka

In [19]:
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")


('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/sentencepiece.bpe.model',
 './final_model/added_tokens.json')

In [20]:
import shutil

shutil.make_archive("/kaggle/working/final_model", 'zip', "./final_model")


'/kaggle/working/final_model.zip'

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")

# Configure 4-bit quantization
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load model and tokenizer from Kaggle input directory
model_path = "./final_model"
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_path,
    quantization_config=quant_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

# Set target language
tgt_lang = "eng_Latn"
forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)
model.config.forced_bos_token_id = forced_bos_token_id
model.config.use_cache = True

# Load Manipuri test sentences
test_file = "/kaggle/input/emnlp-test-dataset-2025/indicMT_testset2025_release/Category-1/Manipuri/mni-en.txt"
with open(test_file, "r", encoding="utf-8") as f:
    manipuri_sentences = [line.split("\t")[0].strip() for line in f if line.strip()]

from tqdm import tqdm  

eng_predictions = []
for sentence in tqdm(manipuri_sentences, desc="Translating"):
    inputs = tokenizer(
        f"mni_Beng {sentence}",
        return_tensors="pt",
        truncation=True,
        max_length=256
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        forced_bos_token_id=forced_bos_token_id,
        max_length=256,
        num_beams=5,
        #early_stopping=True
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    eng_predictions.append(decoded)

# Save predictions
with open("myresult.txt", "w", encoding="utf-8") as f:
    for pred in eng_predictions:
        f.write(pred + "\n")
